In [ ]:
import pandas as pd

file_path = 'data/Job Hunting Tracking.xlsx'

try:
    # Use pandas to read the Excel file
    # This will load the first sheet by default
    df = pd.read_excel(file_path)
    
    df = df.iloc[:, :-3]
    # Print a success message
    print("Successfully loaded the Excel file into a DataFrame!")

    # Display the first 5 rows of the DataFrame to check the data
    print("\nHere are the first 5 rows of your data:")
    print(df.head())

    # Display information about the DataFrame, like column names and data types
    print("\nDataFrame Info:")
    df.info()
    
    df['Job Posting Text'] = df['Job Posting Text'].str.replace(r'\.net', 'dotnet', case=False, regex=True)
    df['Job Posting Text'] = df['Job Posting Text'].str.replace(r'C#', 'csharp', case=False, regex=True)
    df['Job Posting Text'] = df['Job Posting Text'].str.replace(r'C\+\+', 'cpp', case=False, regex=True)
    

except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found.")
    print("Please make sure the Excel file is in the same directory as your Python script, or provide the full path to the file.")
except Exception as e:
    print(f"An error occurred: {e}")



In [ ]:
df['Application Date'] = pd.to_datetime(df['Application Date'])

# 2. Define your start date
start_date = pd.to_datetime('5/13/2025')


filtered_df = df[df['Application Date'] >= start_date]

In [ ]:
filtered_df.head()

In [ ]:
all_postings_text = filtered_df['Job Posting Text']

In [ ]:
all_postings_text.head()

In [ ]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# --------------------------------------------------------------------

# Load English stopwords
stop_words = set(stopwords.words('english'))

def clean_text_for_topic_extraction(text):
    """
    Cleans a single job posting text by:
    1. Lowercasing the text.
    2. Removing punctuation and numbers.
    3. Tokenizing the text into words.
    4. Removing stopwords.
    5. Joining the cleaned words back into a string.
    """
    if not isinstance(text, str):
        return "" # Return empty string for non-string inputs (e.g., NaN)

    # 1. Lowercase the text
    text = text.lower()

    # 2. Remove punctuation and numbers
    # This regex keeps only letters and spaces
    text = re.sub(r'[^a-z\s]', '', text)

    # 3. Tokenize the text
    words = word_tokenize(text)

    # 4. Remove stopwords
    filtered_words = [word for word in words if word not in stop_words and len(word) > 1] # Also remove single character words

    # 5. Join the cleaned words back into a string
    return " ".join(filtered_words)

# Apply the cleaning function to your 'Job Posting Text' series
# Using .apply() is efficient for Series operations
cleaned_job_posting_series = all_postings_text.apply(clean_text_for_topic_extraction)

print("Original 'Job Posting Text' (head):")
print(all_postings_text.head())
print("\n" + "="*50 + "\n")
print("Cleaned 'Job Posting Text' (head after stopword removal):")
print(cleaned_job_posting_series.head())


In [ ]:
# Define the canonical name and a list of variations to search for.
# The search is case-insensitive, so we use lowercase.
# We also handle special characters like '.' in '.net'.

TECH_KEYWORDS = {
    # Cloud & DevOps
    'aws': ['aws', 'amazon web services'],
    'azure': ['azure', 'microsoft azure'],
    'gcp': ['gcp', 'google cloud'],
    'docker': ['docker'],
    'kubernetes': ['kubernetes', 'k8s'],
    'terraform': ['terraform'],
    'jenkins': ['jenkins'],
    'kafka': ['kafka', 'apache kafka'],

    # Backend Languages & Frameworks
    'python': ['python'],
    'java': ['java'],
    'c#': ['csharp', '.net', 'dotnet'],
    'c++': ['cpp'],
    'go': ['go', 'golang'],
    'ruby': ['ruby'],
    'php': ['php'],
    'node.js': ['node.js', 'nodejs'],
    'django': ['django'],
    'flask': ['flask'],
    'spring': ['spring', 'spring boot'],
    'laravel': ['laravel'],

    # Frontend Languages & Frameworks
    'javascript': ['javascript', 'js'],
    'typescript': ['typescript', 'ts'],
    'react': ['react', 'reactjs', 'react.js'],
    'angular': ['angular', 'angularjs'],
    'vue.js': ['vue', 'vuejs', 'vue.js'],
    'html': ['html'],
    'css': ['css'],

    # Databases
    'sql': ['sql'],
    'mysql': ['mysql'],
    'postgresql': ['postgresql', 'postgres'],
    'mongodb': ['mongodb'],
    'redis': ['redis'],

    # Data Science & AI
    'pandas': ['pandas'],
    'numpy': ['numpy'],
    'scikit-learn': ['scikit-learn', 'sklearn'],
    'tensorflow': ['tensorflow'],
    'pytorch': ['pytorch'],
    'machine learning': ['machine learning', 'ml'],
}


In [ ]:
from collections import Counter


# --------------------------------------------------------------------


# Use collections.Counter for efficient counting
tech_counter = Counter()

# Iterate through each cleaned job posting text
for text in cleaned_job_posting_series:
    # Use a set to track skills found in this specific post to avoid double-counting
    found_in_this_post = set()

    # Check for each canonical technology
    for canonical_name, variations in TECH_KEYWORDS.items():
        # Check all variations for that technology
        for variation in variations:
            # Use regex to find whole words only to avoid partial matches (e.g., 'java' in 'javascript')
            # The `\b` is a word boundary.
            if re.search(r'\b' + re.escape(variation) + r'\b', text):
                found_in_this_post.add(canonical_name)
                break # Move to the next canonical name once a variation is found

    # Update the main counter with the unique skills found in this post
    tech_counter.update(found_in_this_post)

# Convert the counter to a pandas DataFrame for better viewing and sorting
tech_freq_df = pd.DataFrame(tech_counter.items(), columns=['Technology', 'Frequency'])
tech_freq_df = tech_freq_df.sort_values(by='Frequency', ascending=False).reset_index(drop=True)

# --- Display the results ---
print("Frequency of Technical Skills Mentioned in Job Postings:")
print(tech_freq_df)


In [ ]:
# Convert the counter to a pandas DataFrame for better viewing and sorting
tech_freq_df = pd.DataFrame(tech_counter.items(), columns=['Technology', 'Frequency'])
tech_freq_df = tech_freq_df.sort_values(by='Frequency', ascending=False).reset_index(drop=True)

# --- Display the results ---
print("Frequency of Technical Skills Mentioned in Job Postings:")
print(tech_freq_df)

# --- SAVE THE OUTPUT TO A CSV FILE (New line) ---
tech_freq_df.to_csv('data/technical_skills_frequency.csv', index=False)

print("\nSuccessfully saved the frequency count to 'technical_skills_frequency.csv'")